# Assignment 3
All questions are weighted the same in this assignment. This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. All questions are worth the same number of points except question 1 which is worth 20% of the assignment grade.

**Note**: Questions 2-13 rely on your question 1 answer.

In [50]:
import pandas as pd
import numpy as np
from functools import reduce

# Filter all warnings. If you would like to see the warnings, please comment the two lines below.
import warnings
warnings.filterwarnings('ignore')

### Question 1
Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **Energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (**Note: there are 1,000,000 gigajoules in a petajoule**). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.

Next, load the GDP data from the file `assets/world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank".*

In [51]:
# Load the energy data from the file `assets/Energy Indicators.xls`
# exclude the footer and header information from the datafile.
# The first two columns are unneccessary,so you should get rid of them, 
# and you should change the column labels so that the columns are:
# `['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`
# `Energy Supply` to gigajoules (**Note: there are 1,000,000 gigajoules in a petajoule**). 
# For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

# Rename the following list of countries (for use in later questions):
# ```"Republic of Korea": "South Korea",
# "United States of America": "United States",
# "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
# "China, Hong Kong Special Administrative Region": "Hong Kong"```
# Be sure to remove the parenthesis, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.

# Next, load the GDP data from the file `assets/world_bank.csv`, which is a csv. Call this DataFrame **GDP**. 
# Make sure to skip the header, and rename the following list of countries:
# ```"Korea, Rep.": "South Korea", 
# "Iran, Islamic Rep.": "Iran",
# "Hong Kong SAR, China": "Hong Kong"```

# Next, load the file `assets/scimagojr-3.xlsx`, which ranks countries based on their journal contributions
# Call this DataFrame **ScimEn**.

# Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). 
# Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15)
# *This function should return a DataFrame with 20 columns and 15 entries, 
# and the rows of the DataFrame should be sorted by "Rank".*
def clean_energy():
    energy_data = pd.read_excel(r'./assets/Energy Indicators.xls',
                                skiprows=16,
                                skipfooter=53,
                                usecols=[2,3,4,5,6])
    Energy = pd.DataFrame(data=energy_data)
    Energy.rename(columns={"Unnamed: 2": "Country", 
                           "Energy Supply":"Energy Supply",
                           "Energy Supply per capita":"Energy Supply per Capita", 
                           "Renewable Electricity Production":"% Renewable"}, 
                  inplace=True)
    Energy.drop(Energy.index[0], 
                inplace=True)
    Energy["Energy Supply"] = Energy["Energy Supply"] * 1000000
    Energy = Energy.replace(['\.\.\.'], 
                            np.nan,
                            regex=True)
    country_name_map = {"Republic of Korea": "South Korea",
                        "United States of America": "United States",
                        "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                        "China, Hong Kong Special Administrative Region3": "Hong Kong"}
    list_keys = (list(country_name_map.keys()))
    list_values = (list(country_name_map.values()))
    Energy.replace(list_keys, list_values, inplace=True)
    Energy['Country']=Energy['Country'].str.replace('\(.*','').str.strip()
    return Energy

def clean_gdp():
    gdp_data = pd.read_csv('./assets/world_bank.csv', skiprows=4)
    GDP = pd.DataFrame(gdp_data)
    GDP.rename(columns={"Country Name": "Country"},
               inplace=True)
    GDP.replace(["Korea, Rep.",
                 "Iran, Islamic Rep.",
                 "Hong Kong SAR, China" ],
                ["South Korea",
                 "Iran",
                 "Hong Kong"],
                inplace=True)
    return GDP

def clean_scimen():
    scimen_data = pd.read_excel('./assets/scimagojr-3.xlsx')
    ScimEn = pd.DataFrame(scimen_data)
    return ScimEn
    
def answer_one():
    Energy = clean_energy()
    GDP = clean_gdp()
    GDP = GDP.loc[:,['Country',
                     '2007','2008','2009','2010','2011','2012','2013','2014','2015']]
    ScimEn = clean_scimen()
    df_list = [Energy, GDP, ScimEn[ScimEn.Rank <= 15]]
    first_merge = pd.merge(Energy,
                          GDP,
                          on='Country',
                          how='right')
    final_merge = pd.merge(ScimEn[ScimEn.Rank <=15],
                            first_merge,
                          on='Country',
                          how='left')
#     print(final_merge)
    return final_merge
answer_one()

,Rank,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,1,China,127050,126767,597237,411683,4.70,138,NaN,NaN,NaN,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
1,2,United States,96661,94747,792274,265436,8.20,230,NaN,NaN,NaN,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
2,3,Japan,30504,30287,223024,61554,7.31,134,NaN,NaN,NaN,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
3,4,United Kingdom,20944,20357,206091,37874,9.84,139,NaN,NaN,NaN,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
4,5,Russian Federation,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.288680,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
5,6,Canada,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.945430,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
6,7,Germany,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.901530,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
7,8,India,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.969080,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
8,9,France,13153,12973,130632,28601,9.93,114,NaN,NaN,NaN,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12
9,10,South Korea,11983,11923,114675,22595,9.57,104,1.100700e+10,221.0,2.279353,9.924316e+11,1.020510e+12,1.027730e+12,1.094499e+12,1.134796e+12,1.160809e+12,1.194429e+12,1.234340e+12,1.266580e+12


In [52]:
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"

assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"


In [53]:
# Cell for autograder.


### Question 2
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [54]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [55]:
def answer_two():
    GDP = clean_gdp()
    Energy = clean_energy()
    Sci = clean_scimen()
    union_one = pd.merge(Sci, Energy, on='Country', how='outer')
    full_union = pd.merge(union_one, GDP,on='Country', how='outer')
    intersect_one = pd.merge(Sci, Energy, on='Country', how='inner')
    all_intersect = pd.merge(intersect_one, GDP, on='Country', how='inner')
    lost_entries = len(full_union) - len(all_intersect)
    return lost_entries

answer_two()

191

In [56]:
assert type(answer_two()) == int, "Q2: You should return an int number!"


### Question 3
What are the top 15 countries for average GDP over the last 10 years?

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [57]:
def answer_three():
    GDP = clean_gdp()
    GDP = GDP.loc[:,['Country',
                     '2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']]
    GDP.set_index('Country', inplace=True)
    GDP.drop(labels=['World', "High income", 'OECD members', 'Post-demographic dividend',
              'IDA & IBRD total', 'Europe & Central Asia', 'Low & middle income', 'Middle income',
             'IBRD only', 'European Union', 'East Asia & Pacific', 'North America', 'Upper middle income', 
              'Late-demographic dividend', 'Euro area', 'Early-demographic dividend', 
              'East Asia & Pacific (excluding high income)','East Asia & Pacific (IDA & IBRD countries)',
             'Latin America & Caribbean','Lower middle income','Latin America & Caribbean (excluding high income)',
             'Europe & Central Asia (IDA & IBRD countries)','Europe & Central Asia (excluding high income)',
              'Middle East & North Africa','Caribbean (IDA & IBRD countries)','South Asia (IDA & IBRD)',
              'Latin America & the Caribbean (IDA & IBRD countries)','Latin America & the Caribbean (IDA & IBRD countries)',
              'South Asia', 'Arab World', 'IDA total','Sub-Saharan Africa','Sub-Saharan Africa (IDA & IBRD countries)',
              'Sub-Saharan Africa (excluding high income)','Central Europe and the Baltics','Middle East & North Africa (excluding high income)',
              'Middle East & North Africa (IDA & IBRD countries)','Pre-demographic dividend'],
             axis=0,
             inplace=True,
            errors='ignore')
#     GDP["average"] = GDP.drop(['Country'], axis=1).mean(axis=1)
    GDP['average'] = GDP.mean(axis=1)
    GDP = GDP.sort_values(by=['average'], ascending=False)
    fifteen = GDP.iloc[:15, [10]]
    avgGDP = pd.Series(fifteen.index, index=fifteen['average'])
    return avgGDP
    raise NotImplementedError()
answer_three()

average
1.536434e+13         United States
6.348609e+12                 China
5.542208e+12                 Japan
3.493025e+12               Germany
2.681725e+12                France
2.487907e+12        United Kingdom
2.189794e+12                Brazil
2.120175e+12                 Italy
1.769297e+12                 India
1.660647e+12                Canada
1.565459e+12    Russian Federation
1.418078e+12                 Spain
1.164043e+12             Australia
1.106715e+12           South Korea
1.090990e+12                Mexico
Name: Country, dtype: object

In [58]:
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"


### Question 4
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [59]:
def answer_four():
    GDP = clean_gdp()
    GDP.drop(columns=['Country Code', 'Indicator Name', 'Indicator Code'], inplace=True)
    uk = GDP.loc[GDP['Country']=='United Kingdom']
    uk = uk.loc[:,['Country',
                     '2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']]
    answer = int(uk['2015'] - uk['2006'])
    return answer
answer_four()

246702696075

In [60]:
# Cell for autograder.


### Question 5
What is the mean energy supply per capita?

*This function should return a single number.*

In [61]:
def answer_five():
    mean_energy = clean_energy()['Energy Supply per Capita'].mean(axis=0)
    return mean_energy
answer_five()

91.0625

In [62]:
# Cell for autograder.


### Question 6
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [63]:
def answer_six():
    Energy = clean_energy()
    Energy.set_index('Country', inplace=True)
    x = Energy['Renewable Electricity Production'].max()
    y = Energy['Renewable Electricity Production'].idxmax()
    print(Energy.sort_values(by=['Renewable Electricity Production'], ascending=False))
    return (y,x)
#     raise NotImplementedError()
answer_six()

KeyError: 'Renewable Electricity Production'

In [64]:
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"

assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"


KeyError: 'Renewable Electricity Production'

### Question 7
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [65]:
def answer_seven():
    Sci = clean_scimen()
    Sci['ratio'] = Sci['Self-citations'] / Sci['Citations']
    Sci = Sci.sort_values(by=['ratio'], ascending=False)
    country = Sci.iloc[0,[1]].to_string(index=False)
    ratio = Sci.iloc[0,[8]].to_string(index=False)
    answer = (country, ratio)
    return answer
#     Sci['citation ratio'] =
#     raise NotImplementedError()
answer_seven()

(' French Guiana', ' 1')

In [66]:
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"

assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"


### Question 8

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return the name of the country*

In [67]:
def answer_eight():
    Energy = clean_energy()
    Energy['population'] = Energy['Energy Supply'] / Energy['Energy Supply per Capita']
    Energy = Energy.sort_values(by=['population'], ascending=False)
    Energy = Energy.reset_index()
    x = Energy.iloc[[2]]['Country'].to_string(index=False)
    return x
#     raise NotImplementedError()
answer_eight()

' Indonesia8'

In [68]:
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"


### Question 9
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [69]:
def answer_nine():
    Energy = clean_energy()
    Sci = clean_scimen()
    Top = answer_one()
    
    Top['population'] = Top['Energy Supply'] / Top['Energy Supply per Capita']
    Top = Top.sort_values(by='population').dropna()
    
    Top['Citable docs per Capita'] = Top['Citable documents'] / Top['population']
    correlation = Top['Citable docs per Capita'].corr(Top['Energy Supply per Capita'])
    return correlation
answer_nine()

0.884992706443506

In [70]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [71]:
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"


### Question 10
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [72]:
def answer_ten():
    Top = answer_one()
    print(Top['% Renewable'].median())
    Top['Renewable above median'] = np.where(Top['% Renewable'] >= Top['% Renewable'].median(), 1, 0)
    HighRenew = pd.Series(Top['Renewable above median'].values, index=Top['Country'])
    return HighRenew 
answer_ten()

17.28868


Country
China                 0
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                0
South Korea           0
Italy                 0
Spain                 0
Iran                  0
Australia             0
Brazil                1
dtype: int64

In [73]:
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"


17.28868


### Question 11
Use the following dictionary to group the Countries by Continent, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [91]:
def answer_eleven():
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    Top = answer_one()
    Top.set_index('Country', inplace=True)
    Top['Continent'] = Top.index.to_series().map(ContinentDict)
    Top['population'] = Top['Energy Supply']/Top['Energy Supply per Capita']
    Top.set_index('Continent', inplace=True)
    Top = Top.groupby(level=0)['population'].agg({'size': np.size, 'sum':np.sum, 'mean':np.mean, 'std':np.std})
    return Top
#     raise NotImplementedError()
answer_eleven()

,size,sum,mean,std
Continent,,,,
Asia,5.0,1.403612e+09,4.678706e+08,7.006261e+08
Australia,1.0,0.000000e+00,NaN,NaN
Europe,6.0,2.238697e+08,1.119348e+08,4.463987e+07
North America,2.0,3.523986e+07,3.523986e+07,NaN
South America,1.0,2.059153e+08,2.059153e+08,NaN


In [92]:
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"

assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"

assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"


### Question 12
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [ ]:
def answer_twelve():
    # YOUR CODE HERE
    raise NotImplementedError()


In [ ]:
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"

assert len(answer_twelve()) == 9, "Q12: Wrong result numbers!"


### Question 13
Convert the Population Estimate series to a string with thousands separator (using commas). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12,345,678.90

*This function should return a series `PopEst` whose index is the country name and whose values are the population estimate string*

In [95]:
def answer_thirteen():
    Top = answer_one()
    Top['Population'] = Top['Energy Supply']/Top['Energy Supply per Capita']
    print(Top['Population'])
#     raise NotImplementedError()
answer_thirteen()

0              NaN
1              NaN
2              NaN
3              NaN
4     1.435000e+08
5     3.523986e+07
6     8.036970e+07
7     1.276731e+09
8              NaN
9     4.980543e+07
10             NaN
11             NaN
12    7.707563e+07
13             NaN
14    2.059153e+08
Name: Population, dtype: float64


In [ ]:
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"

assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"


### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")